
# COVID-Team-1
### Dr. Mohanty
### Brantley, Kawabata, Urgitus
### CSC 405 01
### Last Updated: 10/13/2020
### We, the members of COVID-Team-1, have abided by the UNCG academic policy.

### Module Imports

In [1]:
import ipynb
from ipynb.fs.full import covid_data as cd
import pandas as pd

### Pandas Display Options

In [2]:
# Telling pandas not convert to html tags.
pd.set_option('display.notebook_repr_html', False)
# Max columns and rows to display.
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 10)

## Calculating Weekly Statistics

In [3]:
# Get covid daily cases.
df_daily_cases = cd.get_covid_daily_cases_normalized()
# Get primary data and group by state.
df_primary = cd.get_covid_primary_data()
df_primary_states = df_primary.groupby("State")
# Calculate the number of columns, i.e. days.
number_days = len(df_primary_states.first().filter(regex = "case").columns)
# Declaring/Initializing our state/week lists which will form our indexes.
state_index = []
week_index = []
# Keeps track of which week we are on.
week_count = 1
# Appends a state to the state index.
# Appends ((number_days) / 7) weeks into week index.
# Populating our lists, which will be used for multi indexing.
for state in df_daily_cases.index:
    for week in range (0, number_days, 7):
        state_index.append(state)
        week_index.append("week " + str(week_count))
        week_count += 1
    week_count = 1
# Creating temporary dataframe to hold mean for cases and deaths.
df_weekly_statistics = pd.DataFrame(index = [state_index, week_index],\
                                   columns = ["mean_cases", "mean_deaths", "start_day"])
# Creating our daily statistics dataframe for cases.
df_statistics_cases = pd.DataFrame(index = [df_daily_cases.index],\
                                   columns = ["mean", "median", "mode"])
# Creating our daily statistics dataframe for deaths.
df_daily_deaths = cd.get_covid_daily_deaths_normalized()
df_statistics_deaths = pd.DataFrame(index = [df_daily_cases.index],\
                                   columns = ["mean", "median", "mode"])
# Calculating weekly mean for cases and deaths.
# Week count starts at 0, it is now used for index positioning.
week_count = 0
for lower_range in range(0, number_days, 7):
    upper_range = lower_range + 7
    for state in df_daily_cases.index:
        # Get weekly statistics for cases.
        df_weekly_statistics.loc[state].iloc[week_count]["mean_cases"]\
            = df_daily_cases.loc[state].iloc[lower_range:upper_range].mean().round(0)
        df_weekly_statistics.loc[state].iloc[week_count]["start_day"]\
            = df_daily_cases.loc[state].index[lower_range]
        # Get weekly statistics for deaths.
        df_weekly_statistics.loc[state].iloc[week_count]["mean_deaths"]\
            = df_daily_deaths.loc[state].iloc[lower_range:upper_range].mean().round(0)
    week_count += 1
# Calculating the mean/median/mode for each state, determined from values from all weeks.
for state in df_weekly_statistics.index.get_level_values(0).unique():
    df_statistics_cases.loc[state, "mean"] = df_weekly_statistics.loc[state]["mean_cases"].mean()
    df_statistics_cases.loc[state, "median"] = df_weekly_statistics.loc[state]["mean_cases"].median()
    df_statistics_cases.loc[state, "mode"] = df_weekly_statistics.loc[state]["mean_cases"].mode()[0]
    df_statistics_deaths.loc[state, "mean"] = df_weekly_statistics.loc[state]["mean_deaths"].mean()
    df_statistics_deaths.loc[state, "median"] = df_weekly_statistics.loc[state]["mean_deaths"].median()
    df_statistics_deaths.loc[state, "mode"] = df_weekly_statistics.loc[state]["mean_deaths"].mode()[0]

### Statistics for Cases by State

In [4]:
display(df_statistics_cases)

          mean median mode
State                     
AK     376.171    156    0
AL     1220.83    770    0
AR      1038.4    573    0
AZ     1200.43    556    0
CA     814.971    578    0
...        ...    ...  ...
VT     111.086     94    0
WA     442.457    413    0
WI     753.343    558    0
WV     330.743    199    0
WY     376.229    205    0

[51 rows x 3 columns]

### Statistics for Deaths by State

In [5]:
display(df_statistics_deaths)

          mean median mode
State                     
AK     2.51429      0    0
AL     20.1143     21    0
AR        27.6      8    0
AZ     31.0571     26    0
CA     15.7714     17    0
...        ...    ...  ...
VT     3.71429      0    0
WA     10.9429     10    0
WI     8.97143     10    0
WV     8.85714      3    0
WY     4.31429      0    0

[51 rows x 3 columns]

## Comparing Weekly Statistics